## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Constitucion,CL,-35.33,-72.42,55.56,76,0,4.00,clear sky
1,1,Cape Town,ZA,-33.93,18.42,78.01,50,0,11.41,clear sky
2,2,Saint George,US,37.10,-113.58,30.20,58,1,3.36,clear sky
3,3,Mataura,NZ,-46.19,168.86,53.01,97,100,5.99,light rain
4,4,Mahibadhoo,MV,3.78,72.97,82.78,78,99,15.99,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                               (city_data_df["Max Temp"] >= min_temp)]
vacation_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Cape Town,ZA,-33.93,18.42,78.01,50,0,11.41,clear sky
4,4,Mahibadhoo,MV,3.78,72.97,82.78,78,99,15.99,overcast clouds
7,7,Singkang,MY,2.10,102.80,78.80,88,40,2.24,light rain
8,8,Vaini,TO,-21.20,-175.20,75.20,94,75,3.36,light rain
12,12,Butaritari,KI,3.07,172.79,81.73,81,0,18.92,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
vacation_df.count()

City_ID        195
City           195
Country        194
Lat            195
Lng            195
Max Temp       195
Humidity       195
Cloudiness     195
Wind Speed     195
Description    195
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_df.dropna()
clean_df.count()

City_ID        194
City           194
Country        194
Lat            194
Lng            194
Max Temp       194
Humidity       194
Cloudiness     194
Wind Speed     194
Description    194
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,78.01,clear sky,-33.93,18.42,
4,Mahibadhoo,MV,82.78,overcast clouds,3.78,72.97,
7,Singkang,MY,78.80,light rain,2.10,102.80,
8,Vaini,TO,75.20,light rain,-21.20,-175.20,
12,Butaritari,KI,81.73,clear sky,3.07,172.79,
15,Pangkalanbuun,ID,82.81,light rain,-2.68,111.62,
21,Mahebourg,MU,82.40,shower rain,-20.41,57.70,
34,Teahupoo,PF,80.60,light rain,-17.83,-149.27,
38,Hermanus,ZA,79.00,scattered clouds,-34.42,19.23,
41,Busselton,AU,82.00,clear sky,-33.65,115.33,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           194
Country        194
Max Temp       194
Description    194
Lat            194
Lng            194
Hotel Name     194
dtype: int64

In [10]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Cape Town,ZA,78.01,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town
4,Mahibadhoo,MV,82.78,overcast clouds,3.78,72.97,Omadhoo Inn
7,Singkang,MY,78.80,light rain,2.10,102.80,Kampung Seri Makmor Pagoh
8,Vaini,TO,75.20,light rain,-21.20,-175.20,Keleti Beach Resort
12,Butaritari,KI,81.73,clear sky,3.07,172.79,Isles Sunset Lodge


In [11]:
# 8a. Create the output File (CSV)
export_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(export_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
    
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))